In [81]:
print("5.2 Select DM Methods")

5.1 Match and Discuss DM Methods


In [82]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('infosys722_step2').getOrCreate()

In [83]:
dataset = spark.read.load("./Waste Water Treatment - Transformated.csv", format="csv", header="true")
dataset.show()

+---------+---------+----+---------------+---------------+-------------------+
| Variable|  Country|Year|PercentageValue|Water_quality_T|Life_satisfaction_T|
+---------+---------+----+---------------+---------------+-------------------+
|TOTPUBSEW|Australia|2010|          92.79|     satisfying|              maybe|
|TOTPUBSEW|Australia|2011|          93.84|     satisfying|              maybe|
|TOTPUBSEW|Australia|2012|           94.1|     satisfying|              maybe|
|TOTPUBSEW|Australia|2013|          94.08|     satisfying|              maybe|
|TOTPUBSEW|Australia|2014|          92.57|     satisfying|              maybe|
|TOTPUBSEW|Australia|2015|          92.38|     satisfying|              maybe|
|TOTPUBSEW|Australia|2016|          92.48|     satisfying|              maybe|
|TOTPUBSEW|Australia|2017|          92.69|     satisfying|              maybe|
|TOTPUBSEW|Australia|2018|          92.64|     satisfying|              maybe|
|TOTPUBSEW|Australia|2019|        92.8561|     satis

In [85]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

dataset.columns

['Variable',
 'Country',
 'Year',
 'PercentageValue',
 'Water_quality_T',
 'Life_satisfaction_T']

In [91]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
dataset = dataset.withColumn("Year", dataset["Year"].cast(IntegerType()))
dataset = dataset.withColumn("PercentageValue", dataset["PercentageValue"].cast(FloatType()))

dataset.printSchema()

root
 |-- Variable: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- PercentageValue: float (nullable = true)
 |-- Water_quality_T: string (nullable = true)
 |-- Life_satisfaction_T: string (nullable = true)



In [93]:
assembler = VectorAssembler(
  inputCols=['Year',
             'PercentageValue'],
              outputCol="features")

In [100]:
output = assembler.transform(dataset)
output.show()

+---------+---------+----+---------------+---------------+-------------------+--------------------+
| Variable|  Country|Year|PercentageValue|Water_quality_T|Life_satisfaction_T|            features|
+---------+---------+----+---------------+---------------+-------------------+--------------------+
|TOTPUBSEW|Australia|2010|          92.79|     satisfying|              maybe|[2010.0,92.790000...|
|TOTPUBSEW|Australia|2011|          93.84|     satisfying|              maybe|[2011.0,93.839996...|
|TOTPUBSEW|Australia|2012|           94.1|     satisfying|              maybe|[2012.0,94.099998...|
|TOTPUBSEW|Australia|2013|          94.08|     satisfying|              maybe|[2013.0,94.080001...|
|TOTPUBSEW|Australia|2014|          92.57|     satisfying|              maybe|[2014.0,92.569999...|
|TOTPUBSEW|Australia|2015|          92.38|     satisfying|              maybe|[2015.0,92.379997...|
|TOTPUBSEW|Australia|2016|          92.48|     satisfying|              maybe|[2016.0,92.480003...|


In [97]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Year", outputCol="YearIndex")
output_fixed = indexer.fit(output).transform(output)

In [98]:
final_data = output_fixed.select("features",'YearIndex')

In [99]:
train_data,test_data = final_data.randomSplit([0.7,0.3])